In [1]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

import sys

firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'
#capabilities=firefox_capabilities,
firefox = webdriver.Firefox(executable_path=geckoPath)
#driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')
driver = webdriver.Firefox( executable_path=geckoPath)
try:
    firefox.get('https://www.crunchbase.com/organization/apple')
except:
    pass
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(2)
driver.set_page_load_timeout(30)

def existence_in_crunchbase(name):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+name
    url_crunchbase = None
    try:
        driver.get(url)
    except KeyboardInterrupt:
        sys.exit()
    except:
        print('timeout of phantomjs')
        pass
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url not in url_crunchbase:
            url_crunchbase = None
    
    return url_crunchbase
            

def extract_company_techcrunch(name,url):
    try:
        firefox.get(url)
    except:
        pass
    founded = None
    employees = None
    company_name = ''
    blocks_dt = []
    blocks_dd = []
    try:
        soup = BeautifulSoup(firefox.page_source, "html.parser")
        blocks_dt = soup.select("div.details dt")
        blocks_dd = soup.select("div.details dd")
        company_name_tag = soup.select_one("#profile_header_heading")
        if company_name_tag == None:
            company_name = ''
        else:
            company_name = company_name_tag.getText()
    except:
        pass

    i = 0
    while company_name.lower() != name.lower() or len(blocks_dt)==0 :
        time.sleep(0.3)
        try:
            soup = BeautifulSoup(firefox.page_source, "html.parser")
            company_name_tag = soup.select_one("#profile_header_heading")
            if company_name_tag:
                company_name = company_name_tag.getText()
            else:
                company_name = ''
            blocks_dt = soup.select("div.details dt")
            blocks_dd = soup.select("div.details dd")
        except KeyboardInterrupt:
            sys.exit()
        except:
            pass
        i=i+1
        if i >= 100:
            print('fail to crawl ', name , ' in crunchbase')
            break

    
    if company_name.lower() == name.lower():
        for index,block in enumerate(blocks_dt):
            if 'Founded' in  block.getText() : 
                founded = blocks_dd[index].getText()
            if 'Employees' in block.getText():
                employees = blocks_dd[index].getText().split('|')[0]
    print( name , founded , employees)
    return founded,employees



In [2]:
import time
import json
import os

class CompaniesManager:
    def __init__(self):
        self.companies = []
        self.articles = []
        self.companies_name = []
        self.load_articles()
        self.load_companies()
        
    def load_articles(self):
        if os.path.isfile('data/raw_articles.json'):
            f = open('data/raw_articles.json')
            self.articles = json.load(f)
            f.close()
            
    def load_companies(self):
        if os.path.isfile('data/raw_companies.json'):
            f = open('data/raw_companies.json')
            self.companies = json.load(f)
            f.close()      
            for company in self.companies:
                name = company["name"]
                self.companies_name.append(name)
        
            
    def extract_companies(self):
        
        for index,article in enumerate(self.articles):
            for company_name in article["companies"]: 
                article_extraInfos = article["extra_infos"]
                for info in article_extraInfos:
                    if info["text"]== company_name:
                        relevance = info["relevance"]
                        count_in_article = info["count"]
                            
                if company_name in self.companies_name :
                    for company in self.companies:
                        if company_name == company["name"] and article["id"] not in company["articles"]: 
                            company["count"] = company["count"]+1
                            company["sentiment"]= company["sentiment"]+article["sentiment"]
                            company["articles"].append(article["id"])
                            extra_infos = {
                                 "id":article["id"] ,
                                 "count_in_article":count_in_article,
                                 "revelance": relevance       
                            }
                            company["extra_infos"].append(extra_infos)
                            
                else:
                    company = {
                        "name": company_name,
                        "dateFound": int(str(time.time()).split('.')[0]),
                        "count":1,
                        "sentiment": article["sentiment"],
                        "articles": [article["id"]],
                        "extra_infos":
                        [
                            {
                             "id": article["id"],
                             "count_in_article":count_in_article,
                             "revelance": relevance
                            }  
                        ]    
                    }
                    self.companies.append(company)
                    self.companies_name.append(company_name)
            
        self.save_to_disk()
        
    def extend_crunch(self):
        
        i = 0
        for index,company in enumerate(self.companies):
            name = company["name"]
            search_label = "0"
            if "search_label" in company:
                search_label = company["search_label"]
                
            if search_label=="0" and index >3250:
                url_crunchbase = existence_in_crunchbase(name)
                
                if url_crunchbase != None:
                    print(name, ' in crunchbase')
                    i=i+1
                    founded,employees = extract_company_techcrunch(name,url_crunchbase)
                    company["search_label"] = url_crunchbase
                    if founded is not None:
                        company["foundationDate"] = founded
                    if employees is not None:
                        company["number_of_employees"] = employees
                    
                else:
                    company["search_label"] = str(0)
                    print(name, 'not in crunchbase')
            else:
                print("skip: ", name)
            
            if (index+1)%100 == 0:
                self.save_to_disk()
        self.save_to_disk()            
                                
    def save_to_disk(self):
        with open('data/raw_companies.json', 'w') as company_file:
            json.dump(self.companies, company_file,indent = 2)
            company_file.close()
                
                
                
            
    

In [3]:
CM = CompaniesManager()
#CM.extract_companies()
CM.extend_crunch()


skip:  SENSORO
skip:  Microsoft Corp
skip:  Sumitomo
skip:  Manchester City Verve
skip:  SalesWings
skip:  SA tech
skip:  Avito
skip:  Tshimologong Innovation Hub
skip:  Flickr
skip:  Niantic Labs
skip:  Oblong
skip:  Osterhout Design Group
skip:  Niantic
skip:  Oblong Industries
skip:  Mezzanine
skip:  The Pokemon Company
skip:  GamesBeat
skip:  BNJ Marketing
skip:  DormaKaba
skip:  Berkshire companies
skip:  Bezos
skip:  Private Equity
skip:  Utrip
skip:  Greylock
skip:  Elsight
skip:  LiveU
skip:  WAN
skip:  Vodaphone
skip:  Optus
skip:  GoRemote
skip:  Agri Alliance
skip:  Bitbond
skip:  Obotritia
skip:  FiveAI
skip:  GuestReady
skip:  Carspring
skip:  Moneybox
skip:  Drivetribe
skip:  Realla
skip:  Habito
skip:  Sling & Stone
skip:  Crowdcube
skip:  Cali Rice
skip:  Nominet Trust
skip:  3D Robotics
skip:  AgFunder
skip:  Airware
skip:  Arbe Robotics
skip:  Zipline International
skip:  Redbird
skip:  Bloomberg Businessweek
skip:  Yuneec Electric Aviation
skip:  CyPhy Works
skip:  E

skip:  Rambus
skip:  SiFive
skip:  Citymapper
skip:  Microsemi
skip:  Altera
skip:  Cadence Design Systems
skip:  United Design Systems
skip:  Linley Group
skip:  Synopsys
skip:  Atmel
skip:  HP Enterprise
skip:  Juniper
skip:  livestreaming
skip:  Deezer
skip:  Ultra
skip:  Goldenvoice
skip:  fiat
skip:  PST Consulting
skip:  TGPR
skip:  NOLs
skip:  JumpStart Inc.
skip:  Fog Creek Software
skip:  Allied Gear
skip:  Internet Corporation
skip:  Continuum Analytics
skip:  Sertain Research
skip:  MIT Media Lab
skip:  StreamFuzion Corp
skip:  Bot Insight
skip:  Le Taxi
skip:  Credit Karma
skip:  AIstartups
skip:  Humu
skip:  Greyball
skip:  Shearman & Sterling LLP
skip:  Shearman
skip:  DeepMap
skip:  TetraVue
skip:  Cruise Automation
skip:  Silicon Valley-based
skip:  Proofpoint
skip:  FireLayers
skip:  Twistlock
skip:  Omniture
skip:  Fastdata.io
skip:  FDIO Engine
skip:  Abundant Robotics
skip:  Abundant Robotic
skip:  GV
skip:  Yamaha Motor Company
skip:  BayWa AG
skip:  KPCB Edge
skip

skip:  Flipboard
skip:  TurboPatent
skip:  ARRIS Group Inc
skip:  Digeo
skip:  Melodeo
skip:  Fashwire
skip:  Migo
skip:  Car2go
skip:  Daimler AG
skip:  Maveron
skip:  Virtuix
skip:  Flying Fish
skip:  ReplyYes
skip:  Indix
skip:  Viro Media
skip:  AVX Partners
skip:  HPE
skip:  Career Insights
skip:  Loftium
skip:  Planetary Resources
skip:  Draper Associates
skip:  Branch
skip:  Coastline Market
skip:  Siren
skip:  NEW INC
skip:  Match.com
skip:  Crelate
skip:  Ascentium
skip:  Avanade
skip:  Puzzazz
skip:  Heptio
skip:  Nohla Therapeutics
skip:  Arivale
skip:  InfoSpace
skip:  Snowflake
skip:  Snowflake Computing
skip:  Azure Data Warehouse
skip:  GigaOm
skip:  Qumulo
skip:  Isilon Systems
skip:  Qumulo Core
skip:  Tyche Partners
skip:  Sinclair Oil
skip:  Ripl
skip:  RealNetworks
skip:  Smilebox
skip:  Kauffman
skip:  EveryMove
skip:  higi
skip:  High Fidelity
skip:  Limitless
skip:  Bungie
skip:  ToyTalk
skip:  GrubHub
skip:  Intelius
skip:  Dwellable
skip:  Versium
skip:  Talent

skip:  Whispir
skip:  Adore Beauty
skip:  Borderfree Retail
skip:  Pitney Bowes
skip:  Tmall
skip:  Commpow
skip:  Harvey Norman
skip:  Flexischools
skip:  EnergyLab
skip:  Climate-KIC Australia
skip:  Lightning Lab Electric
skip:  CreativeHQ
skip:  UNSW Innovations
skip:  zipPay
skip:  ARCHFASHION
skip:  Snappr
skip:  POSMusic
skip:  QSIC
skip:  TheCarTest
skip:  Suzuki
skip:  MG Motors
skip:  Winkd
skip:  Vampr
skip:  Seek
skip:  Hudson
skip:  Fixit
skip:  agtech
skip:  SeedRock Africa Agriculture
skip:  Findex
skip:  Breeding Diagnostics
skip:  MilkFlow
skip:  The Gift Concierge Co
skip:  Employment Hero
skip:  Loop11
skip:  Sendle
skip:  Genesis Energy
skip:  Creative HQ
skip:  Callaghan Innovation
skip:  Zencastr
skip:  Auphonic
skip:  GetSwift
skip:  CIMIC Limited
skip:  QSRH
skip:  ACMI
skip:  Runway Geelong
skip:  Deakin
skip:  TechSydney
skip:  Assi
skip:  CSIRO Innovation Fund
skip:  Doshii
skip:  Impos
skip:  NSW Innovation Concierge
skip:  Primary Industries
skip:  SmartBoo

skip:  Balaresque
skip:  Zipcar
skip:  iconiContent
skip:  Cimagine Media
skip:  Flights and Services
skip:  Orbitz
skip:  Kayak
skip:  JetSmarter
skip:  AbilityMate
skip:  Psykinetic
skip:  Seven Sense Technologies
skip:  Lechal
skip:  Counterpoint Research
skip:  Product Hunt
skip:  AngelList
skip:  Hoover
skip:  AngelLists
skip:  Wilson Sonsini Goodrich & Rosati
skip:  PitchBook Data
skip:  Folloze
skip:  Touch Surgery
skip:  Byju
skip:  Raveendran
skip:  Zoho
skip:  Gaussian Networks
skip:  Freshdesk
skip:  Farfetch
skip:  Yoox Net-A-Porter Group
skip:  Plant Labs
skip:  Australian Securities Exchange
skip:  Aquabotix
skip:  Droneshield
skip:  Mu Sigma
skip:  InModi
skip:  Rentberry
skip:  Bozzuto Group
skip:  Grist
skip:  Tiny Rebel Brewery
skip:  Urban IPA
skip:  Wire Graviky Labs
skip:  Graviky Labs
skip:  Infusionsoft
skip:  Clate Mask
skip:  Ernst & Young Entrepreneur
skip:  eight-time Inc.
skip:  Pixloo
skip:  Hostt
skip:  FanDuel
skip:  Trail Answers
skip:  eBay India
skip: 

Opto not in crunchbase
Opto Air not in crunchbase
Virtuali-Tee not in crunchbase
skip:  Imperial Create Lab
Wizdish not in crunchbase
skip:  Worldpay
Softcat not in crunchbase
TFL not in crunchbase
Starwood Hotels & Resorts not in crunchbase
skip:  Open Energi
skip:  Nutmeg
skip:  Tide
First Direct not in crunchbase
skip:  TransferWise
skip:  Atom
Wealthify not in crunchbase
skip:  Mondo
skip:  Baillie Gifford
skip:  Oak Investment Partners
skip:  Addison Lee
CNBC Digital not in crunchbase
skip:  Hailo
skip:  BlaBlaCar
skip:  MyTaxi
skip:  Yandex
Bloomberg Technology not in crunchbase
skip:  Kabbee
Lux services not in crunchbase
Singapore Chinese not in crunchbase
Virtual Commercial not in crunchbase
skip:  Movebubble
skip:  Rightmove
Cowans Nested not in crunchbase
skip:  Foxtons
skip:  Propoly
skip:  GoCardless
skip:  Pi Labs
Healthtech Digital not in crunchbase
ICT Digital not in crunchbase
Customs Digital not in crunchbase
Adtech Digital not in crunchbase
Edtech not in crunchbase
H

Melia Robinson Robots not in crunchbase
Zume not in crunchbase
Bonsai  in crunchbase
Bonsai February 18, 2014 11 - 50 
Keen Browne not in crunchbase
Boehmke not in crunchbase
Part Deux not in crunchbase
General Motor not in crunchbase
Pearl  in crunchbase
Pearl 2016 101 - 250 
theSkimm not in crunchbase
Facebook. not in crunchbase
Acast  in crunchbase
Acast October 10, 2013 51 - 100 
MySpace  in crunchbase
MySpace August 1, 2003 251 - 500 
Reebok not in crunchbase
Audioboom  in crunchbase
Audioboom None None
Edge Investments  in crunchbase
Edge Investments Unknown None
Sonr not in crunchbase
Fuji not in crunchbase
Associated Press  in crunchbase
Associated Press January 1, 1846 5k - 10k 
Levandowski not in crunchbase
Otto Trucking not in crunchbase
leapfrogging not in crunchbase
Vote.org  in crunchbase
Vote.org 1996 None
Akkiraju not in crunchbase
InsiderCisco not in crunchbase
Alcatel-Lucent  in crunchbase
Alcatel-Lucent November 30, 2006 33 in Crunchbase
Versa Networks  in crunchbase

Freshdesk Australia Sreelesh Pillai not in crunchbase
Media Intelligence Co not in crunchbase
Xped not in crunchbase
Lenze Technology not in crunchbase
Solekai Systems  in crunchbase
Solekai Systems 2002 51 - 100 
Solekai not in crunchbase
Aglo not in crunchbase
Outcome.Life not in crunchbase
LiveOffer not in crunchbase
ASX-listed Property Connect not in crunchbase
RentBerry  in crunchbase
RentBerry August 28, 2015 11 - 50 
NFPs not in crunchbase
GiveEasy not in crunchbase
Cerebral Palsy Alliance not in crunchbase
Showtime  in crunchbase
Showtime 1976 101 - 250 
Transformify  in crunchbase
Transformify May 16, 2015 2 in Crunchbase
kompany  in crunchbase
kompany February 29, 2012 15 in Crunchbase
Aura Herbal not in crunchbase
DNV GL  in crunchbase
DNV GL 1864 10k + 
Aurora  in crunchbase
Aurora April 5, 2012 None found in Crunchbase
PriceWaterhouseCoopers not in crunchbase
Dexter Shoe Co. not in crunchbase
Maine Media Collective not in crunchbase
The Knack Factory not in crunchbase
Cowl

In [4]:
CM.save_to_disk()